# Object Detection with YOLO
- YOLO is short for You Only Look Once.
- It is a family of single-stage deep learning-based object detectors.
- They are capable of more than real-time object detection with state-of-the-art accuracy.

## Steps to  be followed:

1. Import the necessary libraries
2. Define the hyperparameter values
3. Define a helper function to download files
4. Pull the data from Roboflow
5. Clone the **YOLOv5** repository
6. Create a directory to store results
7. Run the model
8. Define a function to show validation predictions saved during training
9. Define a helper function for inference on images
10. Visualize inference images






### Step 1: Import the necessary libraries
- Import the **os** module for operating system-related functionalities.
- Import the **glob** module for file path pattern matching.
- Import the **matplotlib.pyplot** module for plotting.
- Import the **OpenCV** module for image processing.
- Import the **requests** module for making HTTP requests.


In [ ]:
import os
import glob as glob
import matplotlib.pyplot as plt
import cv2
import requests


### Step 2: Define the hyperparameter values
- Define the number of epochs to train.

In [ ]:
TRAIN = True
EPOCHS = 5

__Observations:__

The code specifies that the training will be conducted over a total of 5 epochs.

### Step 3: Define a helper function to download files
- Download a file from the provided URL and save it with the specified name.
- Send an HTTP GET request to the URL to download the file.
- Write the content of the response to a file using the specified name.
- If the file already exists, display a message indicating that the download has been skipped.


In [ ]:
def download_file(url, save_name):
    if not os.path.exists(save_name):
        file = requests.get(url)
        open(save_name, 'wb').write(file.content)
    else:
        print('File already present, skipping download...')


### About the dataset:
- The dataset comprises images depicting different vehicles in diverse traffic conditions. These images have been sourced from the Open Image dataset.
- In total, the dataset consists of 5 classes, namely: Car, Bus, Motorcycle, Truck, and Ambulance.

### Step 4: Pull the data from Roboflow

-  If the **train** directory doesn't exist, download the dataset zip file from the specified URL using curl.
- Unzip the downloaded file and remove the zip file after extraction.
- Create a list of directory names: **train, valid, test**.
- Iterate over each directory name and its corresponding index.
- Get a sorted list of all image names in the current directory.
- Iterate over each image name and its corresponding index.
- If the index is even (i.e., every second image), extract the file name without the extension.
- Remove the image file.
- Remove the corresponding label file.


In [ ]:
if not os.path.exists('train'):

    !/usr/bin/curl -L "https://public.roboflow.com/ds/xKLV14HbTF?key=aJzo7msVta" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
    dirs = ['train', 'valid', 'test']
    for i, dir_name in enumerate(dirs):
        all_image_names = sorted(os.listdir(f"{dir_name}/images/"))
        for j, image_name in enumerate(all_image_names):
            if (j % 2) == 0:
                file_name = image_name.split('.jpg')[0]
                os.remove(f"{dir_name}/images/{image_name}")
                os.remove(f"{dir_name}/labels/{file_name}.txt")


**Observations:**

The code block downloads a dataset, extracts it, and selectively removes certain image and label files from the directories.

### Step 5: Clone the YOLOv5 repository
- If the **YOLOv5** directory does not exist, clone the **YOLOv5** repository from the provided GitHub URL.
- Change the current working directory to **YOLOv5**.
- Install the necessary Python packages listed in the **requirements.txt** file.


In [ ]:
if not os.path.exists('yolov5'):

    !git clone https://github.com/ultralytics/yolov5.git

%cd yolov5/

!pip3 install -r requirements.txt


/content/yolov5


**Observations:**

The code block checks for the presence of the **YOLOv5** project directory, clones it (if necessary), sets it as the current working directory, and installs the required dependencies.

### Step 6: Create a directory to store results
- Set the directory to store results based on the current number of result directories.
- Count the number of result directories in the **runs/train** directory. If the **TRAIN** variable is **True** (assuming it's defined outside of this function), then set the result directory name with an incremented count.
- Otherwise, set the result directory name without incrementing the count.
- Return the name of the result directory.


In [ ]:
def set_res_dir():
    # Directory to store results
    res_dir_count = len(glob.glob('runs/train/*'))
    print(f"Current number of result directories: {res_dir_count}")
    if TRAIN:
        RES_DIR = f"results_{res_dir_count+1}"
        print(RES_DIR)
    else:
        RES_DIR = f"results_{res_dir_count}"
    return RES_DIR

### Step 7: Run the model
- Set the **RES_DIR** variable by calling the **set_res_dir()** function.
- If the **TRAIN** variable is **True**, execute the **train.py** script with the specified command-line arguments.


In [ ]:
RES_DIR = set_res_dir()
if TRAIN:
    # VOC: python3
    !python3 train.py --data ../data.yaml --weights yolov5s.pt \
    --img 640 --epochs {EPOCHS} --batch-size 16 --name {RES_DIR}



Current number of result directories: 1
results_2
2024-05-18 11:35:36.666752: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-18 11:35:36.666815: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-18 11:35:36.684920: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=../data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=5, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False,

**Observations:**

The code sets a results directory using **set_res_dir()**. If **TRAIN** is **True**, it executes a **YOLOv5** training command with specific arguments, saving the results in the results directory.

### Step 8: Define a function to show validation predictions saved during training
- List the contents of the **runs/train/{RES_DIR}** directory.
- Set the **EXP_PATH** variable as the path to the result directory.
- Get a list of paths to predicted images in the result directory.
- Print the list of predicted image paths.
- Iterate over each predicted image path.
- Read the image using **OpenCV**.
- Create a figure with a specific size.
- Display the image (converting BGR to RGB) and turn off the axes.
- Show the image.


In [ ]:
def show_valid_results(RES_DIR):
    !ls runs/train/{RES_DIR}
    EXP_PATH = f"runs/train/{RES_DIR}"
    validation_pred_images = glob.glob(f"{EXP_PATH}/*_pred.jpg")
    print(validation_pred_images)
    for pred_image in validation_pred_images:
        image = cv2.imread(pred_image)
        plt.figure(figsize=(19, 16))
        plt.imshow(image[:, :, ::-1])
        plt.axis('off')
        plt.show()

### Step 9: Define a helper function for inference on images
- Perform inference on images using the specified result directory.
- Create a directory to store inference results.
- Count the number of inference-detection directories.
- Set the inference-detection directory name with an incremented count.
- Execute the **detect.py** script with the specified command-line arguments.
- Return the name of the inference detection directory.


In [ ]:
def inference(RES_DIR, data_path):

    infer_dir_count = len(glob.glob('runs/detect/*'))
    print(f"Current number of inference detection directories: {infer_dir_count}")
    INFER_DIR = f"inference_{infer_dir_count+1}"
    print(INFER_DIR)

    !python3 detect.py --weights runs/train/{RES_DIR}/weights/best.pt \
    --source {data_path} --name {INFER_DIR}
    return INFER_DIR

### Step 10: Visualize inference images
- Perform inference on images using the specified result directory.
- Create a directory to store inference results.
- Count the number of inference-detection directories.
- Set the inference detection directory name with an incremented count.
- Execute the **detect.py** script with the specified command-line arguments and return the name of the inference detection directory.



In [ ]:
def visualize(INFER_DIR):

    INFER_PATH = f"runs/detect/{INFER_DIR}"
    infer_images = glob.glob(f"{INFER_PATH}/*.jpg")
    print(infer_images)
    for pred_image in infer_images:
        image = cv2.imread(pred_image)
        plt.figure(figsize=(19, 16))
        plt.imshow(image[:, :, ::-1])
        plt.axis('off')
        plt.show()

In [ ]:
show_valid_results(RES_DIR)

Output hidden; open in https://colab.research.google.com to view.

**Observations:**

The code defines a function that retrieves inference images from a specified directory. It loads and visualizes each image using OpenCV and Matplotlib, respectively, and displays them.

#### Download the inference data file
- If the **inference_images** directory doesn't exist, extract the contents of the **inference_data.zip** file.
- Else, print a message indicating that the dataset is already present.


In [ ]:
download_file('https://learnopencv.s3.us-west-2.amazonaws.com/yolov5_inference_data.zip',
              'inference_data.zip')
if not os.path.exists('inference_images'):
    !unzip -q "inference_data.zip"
else:
    print('Dataset already present')

#### Inference on images
- Call the **inference** function with the arguments **RES_DIR** and **inference_images**.
- Assign the output to the variable **IMAGE_INFER_DIR**.


In [ ]:
IMAGE_INFER_DIR = inference(RES_DIR, 'inference_images')

Current number of inference detection directories: 0
inference_1
detect: weights=['runs/train/results_2/weights/best.pt'], source=inference_images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=inference_1, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-312-g1bcd17ee Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7023610 parameters, 0 gradients, 15.8 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/4 /content/yolov5/inference_images/image_1.jpg: 448x640 1 Bus, 6 Cars, 52.6ms
image 2/4 /content/yolov5/inference_images/image_2.jpg: 416x640 2 Cars, 48.1ms
image 3/4 /content/yolov5/inference_images/ima

#### Visualizing the inference results
- Invoke the **visualize** function and pass the variable **IMAGE_INFER_DIR** as an argument.
- The visualize function displays the contents of the directory or file represented by the **IMAGE_INFER_DIR** variable.

In [ ]:
visualize(IMAGE_INFER_DIR)

Output hidden; open in https://colab.research.google.com to view.

**Observations:**

 The content associated with the directory or file is displayed.